In [14]:
import torch
from isegm.model.modeling.segformer import MixVisionTransformer, SegformerHead

In [15]:
Pretrained_B2_C = '/playpen-raid/qinliu/projects/ritm_interactive_segmentation/weights/pretrained/mit_b2_converted.pth'

backbone_params = dict(
    in_channels=6,
    embed_dims=64,
    num_stages=4,
    num_layers=[3, 4, 6, 3],
    num_heads=[1, 2, 5, 8],
    patch_sizes=[7, 3, 3, 3],
    strides=[2, 2, 2, 2],
    sr_ratios=[8, 4, 2, 1],
    out_indices=(0, 1, 2, 3),
    mlp_ratio=4,
    qkv_bias=True,
    drop_rate=0.0,
    attn_drop_rate=0.0,
    drop_path_rate=0.1,
    pretrained=Pretrained_B2_C
)


In [16]:
backbone = MixVisionTransformer(**backbone_params)
# backbone.init_weights()
state_dict = torch.load(Pretrained_B2_C)
# print(state_dict.keys())


In [17]:
# remove keys
# state_dict.pop('layers.0.0.projection.weight')
ori_proj_weight = state_dict['layers.0.0.projection.weight']
state_dict['layers.0.0.projection.weight'] = torch.cat([ori_proj_weight, ori_proj_weight], dim=1)


backbone.load_state_dict(state_dict, True)

<All keys matched successfully>

In [18]:
backbone.load_state_dict(state_dict, True)

<All keys matched successfully>

In [19]:
state_dict = torch.load(Pretrained_B2_C)
print(state_dict.keys())

odict_keys(['layers.0.0.projection.weight', 'layers.0.0.projection.bias', 'layers.0.0.norm.weight', 'layers.0.0.norm.bias', 'layers.1.0.projection.weight', 'layers.1.0.projection.bias', 'layers.1.0.norm.weight', 'layers.1.0.norm.bias', 'layers.2.0.projection.weight', 'layers.2.0.projection.bias', 'layers.2.0.norm.weight', 'layers.2.0.norm.bias', 'layers.3.0.projection.weight', 'layers.3.0.projection.bias', 'layers.3.0.norm.weight', 'layers.3.0.norm.bias', 'layers.0.1.0.norm1.weight', 'layers.0.1.0.norm1.bias', 'layers.0.1.0.attn.attn.in_proj_weight', 'layers.0.1.0.attn.attn.in_proj_bias', 'layers.0.1.0.attn.attn.out_proj.weight', 'layers.0.1.0.attn.attn.out_proj.bias', 'layers.0.1.0.attn.sr.weight', 'layers.0.1.0.attn.sr.bias', 'layers.0.1.0.attn.norm.weight', 'layers.0.1.0.attn.norm.bias', 'layers.0.1.0.norm2.weight', 'layers.0.1.0.norm2.bias', 'layers.0.1.0.ffn.layers.0.weight', 'layers.0.1.0.ffn.layers.0.bias', 'layers.0.1.0.ffn.layers.1.weight', 'layers.0.1.0.ffn.layers.1.bias', 'l

In [20]:
from isegm.model.modeling.transformer_helper.cross_entropy_loss import CrossEntropyLoss

decode_head_params = dict(
    in_channels=[64, 128, 320, 512],
    in_index=[0, 1, 2, 3],
    channels=256,
    dropout_ratio=0.1,
    num_classes=1,
    loss_decode=CrossEntropyLoss(),
    align_corners=False,
)

head = SegformerHead(**decode_head_params)


In [21]:
in_shape = (1, 3, 320, 480)
input_t = torch.randn(in_shape)

additional_shape = (1, 3, 320, 480)
additional_t = torch.randn(additional_shape)

input_t = torch.cat([input_t, additional_t], dim=1)

temp = backbone(input_t)
out_t = head(temp)

In [22]:
print(out_t.shape)

torch.Size([1, 1, 160, 240])
